In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Oct 25 14:22:06 2020

@author: AJCR
"""

import numpy as np
import codecs
import tensorflow as tf
import re
import time
import logging
import os.path
import sys
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from gensim.models import word2vec
from operator import *
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow_addons as tfa
from keras.utils.np_utils import to_categorical

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    

#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
#tf.config.experimental.set_memory_growth(physical_devices[0], True)  

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Buid the data

In [2]:
#with open(r'E:\Doctor\resource\tibet_corpus\sentiment_tibet.txt', 'r') as f:
f=codecs.open('./data/new_sentiment_train.txt','r','utf-8')
fw=codecs.open('./data/data_exp(2stacked).txt','w','utf-8')
# reviews_new = pd.read_csv(r'E:\Doctor\code\mycode\sentiment\data\small_new_sentiment_tibet.txt', header=None)
# labels_new =pd.read_csv(r'E:\Doctor\code\mycode\sentiment\data\small_new_sentiment_tags_tibet.txt', header=None)
#aaa=f.readlines()
#print(len(aaa))
reviews = f.readlines()
for i in range(len(reviews)):
    reviews[i]=reviews[i].replace('\ufeff','')
    reviews[i]=reviews[i].replace('\n','')
    #reviews[i]=reviews[i].replace(' ','་')
    #reviews[i]=reviews[i].replace('།','་')
    reviews[i]=reviews[i].replace('《','་')
    reviews[i]=reviews[i].replace('》','་')
    reviews[i]=reviews[i].replace('“','་')
    reviews[i]=reviews[i].replace('”','་')
    reviews[i]=reviews[i].replace('"','་')
    reviews[i]=reviews[i].replace('（','་')
    reviews[i]=reviews[i].replace('）','་')
    reviews[i]=reviews[i].replace('(','་')
    reviews[i]=reviews[i].replace(')','་')
    reviews[i]=reviews[i].replace('feff','')
    reviews[i]=reviews[i].replace('་་','་')
    #reviews[i]=reviews[i].replace('　','')


    
#reviews=reviews[1:]
f.close()
f=codecs.open('./data/new_sentiment_tag.txt', 'r','utf-8')
labels = f.readlines()
#labels=labels[1:]
for t in range(len(labels)):
    labels[t]=int(labels[t].replace("\n",''))-1
print(len(labels))
i=0
for tt in labels:
    if tt>5:
        print(i,'==',tt)
    i=i+1
#print(labels)
f.close()

29142


In [3]:
print(reviews[0:10])
#labels[0:1000]

['ཁྱོད ཀྱིས བརྩེ་བ ལ བརྟེན ན ཅུང་ཙམ སྐྱིད །', 'འཛམ་གླིང ཡོངས ཐུབ ལ ཅུང་ཙམ དགའ །', 'ཁྱོད ཀྱིས སྡང་སེམས ཡོད ན ཅུང་ཙམ སྐྱིད །', 'འཛམ་གླིང ཡོངས ལ ཅུང་ཙམ སྐྱིད །', 'སྐྱེས་པ འི ལང་ཚོ ནི ཅུང་ཙམ སྐྱིད །', 'གཏིང མི ཟབ་པ ཞིག ཡིན ལ ཅུང་ཙམ སྐྱིད །', 'བུད་མེད ཀྱི ལང་ཚོ འི ཅུང་ཙམ སྐྱིད །', 'མཚོན་རྟགས ནི རིན་ཐང ཡིན་པ ཅུང་ཙམ སྐྱིད །', 'ནོར་འཁྲུལ ངོས་ལེན བྱས ན ཅུང་ཙམ བཟང །', 'ནོར་འཁྲུལ ཕྱེད་ཀ བསྒྱུར ཟིན ཅུང་ཙམ ལེགས །']


# data processing

In [4]:
from string import punctuation

all_text = ' '.join(reviews)
all_text=all_text.replace('\ufeff','')
all_text=all_text.replace('\n','')
#all_text=all_text.replace('།','')
print(all_text[:40])
print(reviews[:10])

for i in range(10):
    all_text=all_text.replace('་་','་')
words = all_text.split(u' ')
print(words[:20])
print(len(words))

ཁྱོད ཀྱིས བརྩེ་བ ལ བརྟེན ན ཅུང་ཙམ སྐྱིད 
['ཁྱོད ཀྱིས བརྩེ་བ ལ བརྟེན ན ཅུང་ཙམ སྐྱིད །', 'འཛམ་གླིང ཡོངས ཐུབ ལ ཅུང་ཙམ དགའ །', 'ཁྱོད ཀྱིས སྡང་སེམས ཡོད ན ཅུང་ཙམ སྐྱིད །', 'འཛམ་གླིང ཡོངས ལ ཅུང་ཙམ སྐྱིད །', 'སྐྱེས་པ འི ལང་ཚོ ནི ཅུང་ཙམ སྐྱིད །', 'གཏིང མི ཟབ་པ ཞིག ཡིན ལ ཅུང་ཙམ སྐྱིད །', 'བུད་མེད ཀྱི ལང་ཚོ འི ཅུང་ཙམ སྐྱིད །', 'མཚོན་རྟགས ནི རིན་ཐང ཡིན་པ ཅུང་ཙམ སྐྱིད །', 'ནོར་འཁྲུལ ངོས་ལེན བྱས ན ཅུང་ཙམ བཟང །', 'ནོར་འཁྲུལ ཕྱེད་ཀ བསྒྱུར ཟིན ཅུང་ཙམ ལེགས །']
['ཁྱོད', 'ཀྱིས', 'བརྩེ་བ', 'ལ', 'བརྟེན', 'ན', 'ཅུང་ཙམ', 'སྐྱིད', '།', 'འཛམ་གླིང', 'ཡོངས', 'ཐུབ', 'ལ', 'ཅུང་ཙམ', 'དགའ', '།', 'ཁྱོད', 'ཀྱིས', 'སྡང་སེམས', 'ཡོད']
329398


In [5]:
# Create your dictionary that maps vocab words to integers here
from collections import Counter
def get_vocab_to_int(words):

    # 统计每个单词出现的次数
    counts = Counter(words)
    print()

    # 按出现次数，从多到少排序
    vocab = sorted(counts, key=counts.get, reverse=True)
    print(len(vocab))
    #print(vocab)
    aa=counts.items()
    aa=sorted(aa,key=itemgetter(1),reverse=True)
    h=[t[0] for t in aa if t[1]>1000]
    print(h)
    # 建立单词到数字的映射，也就是给单词贴上一个数字下标，在网络中用数字标签表示单词
    # 例如，'apple'在网络中就是一个数字，比如是500.
    # 数字标签从 1 开始， 0 作特殊作用（下面会说）
    vocab_to_int = { word : i for i, word in enumerate(vocab, 1)}

    return vocab_to_int

def get_reviews_ints(vocab_to_int, reviews):
    # 将review转换为数字，也就是将review中每个单词，通过vocab_to_int转换为数字
    # 例如，"I love this moive" 可能被转换为 [5 36 45 12354]
    reviews_ints = []
    for each in reviews:
        reviews_ints.append( [ vocab_to_int[word] for word in each.split(u' ')] )

    return reviews_ints

vocab_to_int = get_vocab_to_int(words)
print(vocab_to_int['བོད'])
print(len(vocab_to_int))
# Convert the reviews to integers, same shape as reviews list, but with integers
reviews_ints = get_reviews_ints(vocab_to_int, reviews)
print(reviews[10:12])
print(reviews_ints[10:12])


11122
['།', 'འི', 'དགའ', 'ན', 'ལ', 'མི', 'ཅུང་ཙམ', 'ས', 'ཧ་ཅང', 'ར', 'མ', 'སྡུག', 'སྤྲོ', 'དེ', 'སྐྱིད', 'ནས', 'ནི', 'དང', 'ཡང', 'ཞིག', 'ཀྱི', 'འོ', 'ཤིན་ཏུ', 'གི', 'ཆེས', 'ང', 'དུ', 'ཡག', 'དགའ་བ', 'སེམས', 'ཡིན', 'ཡིན་པ', 'ཡོད', 'ཡོད་པ', 'གྱི', 'མེད', 'བྱེད', 'སུ', 'སྐྱོ', 'ཁྱོད', 'པ', 'སྐྱོ་བ', 'བྱས', 'མེད་པ', 'བཟང']
442
11122
['ཆུ་བོ ཐམས་ཅད རྒྱ་མཚོ ར འབབ ན ཅུང་ཙམ བཟང །', 'རྩིག་པ བ བཙུགས ན ཅུང་ཙམ ཡག །']
[[425, 342, 384, 10, 581, 4, 7, 45, 1], [5434, 54, 1325, 4, 7, 28, 1]]


In [6]:
# Convert labels to 1s and 0s for 'positive' and 'negative'
#labels =  np.array([0 if label=='negative' else 1 for label in labels.split('\n')])
labels =  np.array(labels)

In [7]:
from collections import Counter
#print([len(x) for x in reviews_ints])
review_lens = Counter([len(x) for x in reviews_ints])
print(review_lens)
print("Zero-length reviews: {}".format(review_lens[11]))
print("Maximum review length: {}".format(max(review_lens)))

Counter({8: 3121, 9: 3071, 7: 3060, 10: 2566, 11: 2399, 6: 2128, 12: 2008, 13: 1716, 14: 1475, 15: 1229, 16: 1059, 5: 1042, 17: 801, 18: 643, 19: 510, 20: 399, 21: 322, 4: 316, 23: 234, 22: 208, 24: 145, 25: 136, 26: 101, 27: 95, 28: 69, 30: 48, 31: 45, 3: 39, 32: 35, 29: 33, 33: 26, 34: 17, 35: 12, 36: 12, 38: 7, 37: 4, 2: 3, 41: 3, 39: 2, 40: 1, 42: 1, 43: 1})
Zero-length reviews: 2399
Maximum review length: 43


In [8]:
# 获取长度不为0的review的下标
non_zeros_idx = [ ii for ii, review in enumerate(reviews_ints) if len(review) != 0]
print(len(non_zeros_idx))
# Filter out that review with 0 length
reviews_ints = [reviews_ints[ii] for ii in non_zeros_idx]
#print(reviews_ints)


29142


In [9]:
seq_len = 100#200#40#200
print(len(reviews_ints))

features = np.zeros((len(reviews_ints), seq_len), dtype=int)
for i,review in enumerate(reviews_ints):
    #print(review,len(review))
    #print(review[:100],len(review))
    
    features[i, -len(review):] = np.array(review)
    #print(features[i])
    #features[i,:len(review)] = np.array(review)[:seq_len]
print(features[0])

29142
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0  40  48 230   5 207   4   7  15   1]


In [10]:
features[:]
#reviews[:10]


array([[ 0,  0,  0, ...,  7, 15,  1],
       [ 0,  0,  0, ...,  7,  3,  1],
       [ 0,  0,  0, ...,  7, 15,  1],
       ...,
       [ 0,  0,  0, ...,  9,  3,  1],
       [ 0,  0,  0, ...,  9,  3,  1],
       [ 0,  0,  0, ..., 29, 22,  1]])

In [11]:
# 用于训练的比例
def data_process(para):
    split_frac = 0.8
# 将训练集划分出来
    print(features.shape,labels.shape)
    train_x,val_x,train_y,val_y= train_test_split(features, np.array(labels), test_size=0.20, random_state=0)
# 除去训练集，剩下的部分被分为测试集和验证集，一半一半
    test_index = int(len(val_x)*0.5)
    val_x, test_x = val_x[:test_index], val_x[test_index:]
    val_y, test_y = val_y[:test_index], val_y[test_index:]

    print("\t\t\tFeature Shapes:")
    print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))
    
    #wordbed = word2vec.Word2Vec.load("./resource/tibetwordcbowvector.model")
    #embed = tf.nn.embedding_lookup(wordbed, train_x)
   # train_x=wordbed(train_x)
    print(train_x.shape)
    print("aj",train_y[13720:13727])
    return train_x,train_y,val_x,val_y,test_x,test_y
#data_process(para)

In [12]:
import argparse

class Hpara():
    parser = argparse.ArgumentParser()#构建一个参数管理对象
    
    parser.add_argument('--traindatapath',default='./data/train.csv',type=str)
    parser.add_argument('--testdatapath',default='./data/test.csv',type=str)
    
    parser.add_argument('--label2idpath',default='./data/label2id.json',type=str) 
    parser.add_argument('--word2idpath',default='./data/word2id.json',type=str) 
    parser.add_argument('--id2labelpath',default='./data/id2label.json',type=str) 
    parser.add_argument('--id2wordpath',default='./data/id2word.json',type=str) 
    
    parser.add_argument('--testlabel2idpath',default='./data/test_label2id.json',type=str) 
    parser.add_argument('--testword2idpath',default='./data/test_word2id.json',type=str) 
    parser.add_argument('--testid2labelpath',default='./data/test_id2label.json',type=str) 
    parser.add_argument('--testid2wordpath',default='./data/test_id2word.json',type=str)
       
    parser.add_argument('--attenion_drop_rate',default=0.2,type=float)
    parser.add_argument('--attention_size',default=400,type=int)
    parser.add_argument('--is_mask',default=True,type=bool)
    
    parser.add_argument('--max_sen_len',default=100,type=int)
    parser.add_argument('--train_nums',default=39717,type=int)
    parser.add_argument('--test_nums',default=200,type=int)
    parser.add_argument('--word2vector_dim',default=128,type=int)
    parser.add_argument('--hidden_dim',default=400,type=int)
    parser.add_argument('--token_nums',default=20406,type=int)
    parser.add_argument('--label_nums', default=14, type=int)
    parser.add_argument('--rnn_layers_nums',default=2,type=int)
    parser.add_argument('--is_training',default=True,type=bool)
    parser.add_argument('--drop_rate',default=0.1,type=float)    
    parser.add_argument('--learning_rate',default=0.0000001,type=float)
    parser.add_argument('--epochs',default=3000,type=int)
    parser.add_argument('--cell_type',default='LSTM',type=str)
    parser.add_argument('--savepath',default='./check_point',type=str)
    parser.add_argument('--batch_size',default=300,type=int)

In [13]:
class RNN_layer(layers.Layer):
    
    def __init__(self,para):
        super().__init__(self)
        self.para=para
        #self.fw_lstm1=layers.LSTM(self.para.hidden_dim, return_sequences = True, go_backwards= False, dropout = self.para.drop_rate, name = "fwd_lstm")
        #self.bw_lstm1=layers.LSTM(self.para.hidden_dim, return_sequences = True, go_backwards= True, dropout = self.para.drop_rate, name = "bwd_lstm")
        #self.bilstm1=layers.Bidirectional(merge_mode = "concat", layer = self.fw_lstm1, backward_layer = self.bw_lstm1, name = "bilstm")
              
        
        self.bw_lstm=layers.LSTM(self.para.word2vector_dim, return_sequences = True, go_backwards= True, dropout = self.para.drop_rate, name = "bwd_lstm")
        self.fw_lstm=layers.LSTM(self.para.word2vector_dim, return_sequences = True,go_backwards= False, return_state=True,dropout = self.para.drop_rate, name = "fwd_lstm")
        self.fw_lstm1=layers.LSTM(self.para.word2vector_dim, return_sequences = True,go_backwards= False, return_state=True,dropout = self.para.drop_rate, name = "fwd_lstm1")
        #self.fl=layers.Flatten()
        self.fc1=layers.Dense(6,activation=tf.nn.relu)  

    
    def call(self,inputs):
       # outputs=self.bilstm1(inputs,training=self.para.is_training)
        #print(inputs.shape)
        #bw_lstm1=self.bw_lstm(inputs)
        s,_,aa=self.fw_lstm1(inputs)
        #print(bw_lstm1)
        out,_,states=self.fw_lstm(s)
        #out,_,states=self.fw_lstm(bw_lstm1)
        #print(states.shape)
        
        #self.fl=self.fl(states)
        #print(states)
        fc1=self.fc1(states)  
        #print("222222222",fc1)
        
        return fc1        

In [14]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras import layers
from tqdm import tqdm
import matplotlib.pyplot as plt

#from Model_modules import RNN_layer,Self_Attention_Layer,Crf_layer
#rom data_utils import data_process,data_process_ds1
#from Model_para import Hpara



def train_and_test(para):
    train_text,train_label,val_text,val_label,test_text,test_label=data_process(para)
    print('ajcr=',train_label[0:125])
    i=0
    for tt in train_label:
        if tt>5:
            print(i,"=",tt)
        i=i+1
    
    train_label = to_categorical(train_label, 6)
    val_label = to_categorical(val_label, 6)
    test_label = to_categorical(test_label, 6)
    
    model = keras.Sequential()
    embed=layers.Embedding(input_dim=para.token_nums,output_dim=para.word2vector_dim,input_length=para.max_sen_len, name = "embeding")       
    model.add(embed)
    model.add(RNN_layer(para))
    model.summary()
    #model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['categorical_accuracy'])
    
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    history=model.fit(train_text,train_label,epochs=para.epochs,batch_size=para.word2vector_dim,validation_data=(val_text,val_label),verbose=1)
    result=model.evaluate(test_text,test_label)
    #保存模型
    model.save_weights(os.path.join(para.savepath,'ckpt'))
   



    fw.close()
    #下面对模型进行测试

        
if __name__=='__main__':
    hp=Hpara()
    parser = hp.parser
    para = parser.parse_args(args=[])
    train_and_test(para)

(29142, 100) (29142,)
			Feature Shapes:
Train set: 		(23313, 100) 
Validation set: 	(2914, 100) 
Test set: 		(2915, 100)
(23313, 100)
aj [4 0 4 4 0 2 3]
ajcr= [2 0 0 0 1 4 3 1 5 4 5 3 1 1 4 0 3 0 4 5 1 2 0 2 4 1 2 0 4 0 1 3 4 0 0 0 0
 4 1 4 5 0 5 3 5 2 5 5 0 5 1 4 0 1 0 3 4 0 4 0 4 4 4 1 5 1 0 1 2 1 5 1 0 4
 3 4 0 1 5 1 0 0 4 4 2 3 3 3 1 3 3 3 4 4 5 0 2 1 3 3 2 0 3 0 3 5 1 5 2 4 5
 4 0 0 2 3 1 2 0 3 3 2 3 2 4]
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embeding (Embedding)         (None, 100, 128)          2611968   
_________________________________________________________________
rnn_layer (RNN_layer)        (None, 6)                 263942    
Total params: 2,875,910
Trainable params: 2,875,910
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3000
183/183 [==============================] - 10s 57ms/step - loss: 2.3933 - accura

Epoch 50/3000
183/183 [==============================] - 13s 71ms/step - loss: 0.1476 - accuracy: 0.9774 - val_loss: 0.2620 - val_accuracy: 0.9684
Epoch 51/3000
183/183 [==============================] - 13s 71ms/step - loss: 0.0965 - accuracy: 0.9866 - val_loss: 0.2394 - val_accuracy: 0.9691
Epoch 52/3000
183/183 [==============================] - 13s 71ms/step - loss: 0.0866 - accuracy: 0.9889 - val_loss: 0.2316 - val_accuracy: 0.9691
Epoch 53/3000
183/183 [==============================] - 13s 69ms/step - loss: 0.0830 - accuracy: 0.9895 - val_loss: 0.2310 - val_accuracy: 0.9705
Epoch 54/3000
183/183 [==============================] - 13s 70ms/step - loss: 0.0801 - accuracy: 0.9906 - val_loss: 0.2338 - val_accuracy: 0.9708
Epoch 55/3000
183/183 [==============================] - 13s 70ms/step - loss: 0.0778 - accuracy: 0.9904 - val_loss: 0.2360 - val_accuracy: 0.9712
Epoch 56/3000
183/183 [==============================] - 13s 70ms/step - loss: 0.0769 - accuracy: 0.9912 - val_loss: 0

Epoch 106/3000
183/183 [==============================] - 13s 68ms/step - loss: 0.0671 - accuracy: 0.9940 - val_loss: 0.2739 - val_accuracy: 0.9756
Epoch 107/3000
183/183 [==============================] - 12s 68ms/step - loss: 0.0638 - accuracy: 0.9945 - val_loss: 0.2778 - val_accuracy: 0.9760
Epoch 108/3000
183/183 [==============================] - 13s 69ms/step - loss: 0.0613 - accuracy: 0.9946 - val_loss: 0.2743 - val_accuracy: 0.9756
Epoch 109/3000
183/183 [==============================] - 13s 69ms/step - loss: 0.0596 - accuracy: 0.9948 - val_loss: 0.2901 - val_accuracy: 0.9756
Epoch 110/3000
183/183 [==============================] - 13s 69ms/step - loss: 0.0582 - accuracy: 0.9949 - val_loss: 0.2910 - val_accuracy: 0.9756
Epoch 111/3000
183/183 [==============================] - 12s 68ms/step - loss: 0.0575 - accuracy: 0.9952 - val_loss: 0.2879 - val_accuracy: 0.9760
Epoch 112/3000
183/183 [==============================] - 13s 69ms/step - loss: 0.0576 - accuracy: 0.9954 - val_

183/183 [==============================] - 13s 68ms/step - loss: 0.0517 - accuracy: 0.9955 - val_loss: 0.3527 - val_accuracy: 0.9708
Epoch 162/3000
183/183 [==============================] - 13s 69ms/step - loss: 0.0523 - accuracy: 0.9955 - val_loss: 0.3458 - val_accuracy: 0.9715
Epoch 163/3000
183/183 [==============================] - 13s 68ms/step - loss: 0.0520 - accuracy: 0.9959 - val_loss: 0.3446 - val_accuracy: 0.9708
Epoch 164/3000
183/183 [==============================] - 13s 68ms/step - loss: 0.0526 - accuracy: 0.9958 - val_loss: 0.3399 - val_accuracy: 0.9722
Epoch 165/3000
183/183 [==============================] - 12s 68ms/step - loss: 0.0519 - accuracy: 0.9958 - val_loss: 0.3484 - val_accuracy: 0.9715
Epoch 166/3000
183/183 [==============================] - 12s 68ms/step - loss: 0.0520 - accuracy: 0.9956 - val_loss: 0.3586 - val_accuracy: 0.9729
Epoch 167/3000
183/183 [==============================] - 13s 70ms/step - loss: 0.0647 - accuracy: 0.9944 - val_loss: 0.3797 - 

183/183 [==============================] - 13s 69ms/step - loss: 0.1344 - accuracy: 0.9853 - val_loss: 0.3416 - val_accuracy: 0.9688
Epoch 217/3000
183/183 [==============================] - 12s 68ms/step - loss: 0.0871 - accuracy: 0.9878 - val_loss: 0.3486 - val_accuracy: 0.9691
Epoch 218/3000
183/183 [==============================] - 12s 68ms/step - loss: 0.0862 - accuracy: 0.9905 - val_loss: 0.3378 - val_accuracy: 0.9749
Epoch 219/3000
183/183 [==============================] - 13s 69ms/step - loss: 0.0704 - accuracy: 0.9933 - val_loss: 0.3101 - val_accuracy: 0.9743
Epoch 220/3000
183/183 [==============================] - 13s 69ms/step - loss: 0.0621 - accuracy: 0.9946 - val_loss: 0.2990 - val_accuracy: 0.9767
Epoch 221/3000
183/183 [==============================] - 13s 69ms/step - loss: 0.0612 - accuracy: 0.9951 - val_loss: 0.3183 - val_accuracy: 0.9767
Epoch 222/3000
183/183 [==============================] - 12s 68ms/step - loss: 0.0595 - accuracy: 0.9955 - val_loss: 0.3166 - 

...

183/183 [==============================] - 8s 44ms/step - loss: 0.0614 - accuracy: 0.9919 - val_loss: 0.4071 - val_accuracy: 0.9688
Epoch 2747/3000
183/183 [==============================] - 8s 46ms/step - loss: 0.0588 - accuracy: 0.9918 - val_loss: 0.4006 - val_accuracy: 0.9688
Epoch 2748/3000
183/183 [==============================] - 8s 43ms/step - loss: 0.0553 - accuracy: 0.9919 - val_loss: 0.3782 - val_accuracy: 0.9698
Epoch 2749/3000
183/183 [==============================] - 8s 42ms/step - loss: 0.0552 - accuracy: 0.9929 - val_loss: 0.3838 - val_accuracy: 0.9695
Epoch 2750/3000
183/183 [==============================] - 8s 42ms/step - loss: 0.0538 - accuracy: 0.9929 - val_loss: 0.3841 - val_accuracy: 0.9698
Epoch 2751/3000
183/183 [==============================] - 8s 44ms/step - loss: 0.0532 - accuracy: 0.9927 - val_loss: 0.3852 - val_accuracy: 0.9698
Epoch 2752/3000
183/183 [==============================] - 8s 44ms/step - loss: 0.0566 - accuracy: 0.9921 - val_loss: 0.3808 - v

183/183 [==============================] - 8s 44ms/step - loss: 0.0505 - accuracy: 0.9933 - val_loss: 0.3294 - val_accuracy: 0.9729
Epoch 2802/3000
183/183 [==============================] - 8s 46ms/step - loss: 0.0511 - accuracy: 0.9939 - val_loss: 0.3293 - val_accuracy: 0.9729
Epoch 2803/3000
183/183 [==============================] - 8s 46ms/step - loss: 0.0519 - accuracy: 0.9939 - val_loss: 0.3470 - val_accuracy: 0.9722
Epoch 2804/3000
183/183 [==============================] - 8s 45ms/step - loss: 0.0518 - accuracy: 0.9938 - val_loss: 0.3472 - val_accuracy: 0.9725
Epoch 2805/3000
183/183 [==============================] - 8s 43ms/step - loss: 0.0497 - accuracy: 0.9942 - val_loss: 0.3417 - val_accuracy: 0.9725
Epoch 2806/3000
183/183 [==============================] - 8s 44ms/step - loss: 0.0497 - accuracy: 0.9940 - val_loss: 0.3467 - val_accuracy: 0.9729
Epoch 2807/3000
183/183 [==============================] - 8s 44ms/step - loss: 0.0504 - accuracy: 0.9944 - val_loss: 0.3469 - v

183/183 [==============================] - 8s 44ms/step - loss: 0.0435 - accuracy: 0.9936 - val_loss: 0.3561 - val_accuracy: 0.9719
Epoch 2857/3000
183/183 [==============================] - 8s 43ms/step - loss: 0.0455 - accuracy: 0.9932 - val_loss: 0.3561 - val_accuracy: 0.9719
Epoch 2858/3000
183/183 [==============================] - 8s 45ms/step - loss: 0.0414 - accuracy: 0.9935 - val_loss: 0.3615 - val_accuracy: 0.9719
Epoch 2859/3000
183/183 [==============================] - 8s 43ms/step - loss: 0.0435 - accuracy: 0.9939 - val_loss: 0.3616 - val_accuracy: 0.9719
Epoch 2860/3000
183/183 [==============================] - 8s 45ms/step - loss: 0.0455 - accuracy: 0.9927 - val_loss: 0.3617 - val_accuracy: 0.9715
Epoch 2861/3000
183/183 [==============================] - 8s 42ms/step - loss: 0.0449 - accuracy: 0.9937 - val_loss: 0.3389 - val_accuracy: 0.9715
Epoch 2862/3000
183/183 [==============================] - 8s 43ms/step - loss: 0.0462 - accuracy: 0.9931 - val_loss: 0.3389 - v

183/183 [==============================] - 8s 43ms/step - loss: 0.0455 - accuracy: 0.9935 - val_loss: 0.3116 - val_accuracy: 0.9749
Epoch 2912/3000
183/183 [==============================] - 8s 44ms/step - loss: 0.0456 - accuracy: 0.9938 - val_loss: 0.3233 - val_accuracy: 0.9746
Epoch 2913/3000
183/183 [==============================] - 8s 43ms/step - loss: 0.0463 - accuracy: 0.9935 - val_loss: 0.3071 - val_accuracy: 0.9743
Epoch 2914/3000
183/183 [==============================] - 8s 43ms/step - loss: 0.0455 - accuracy: 0.9932 - val_loss: 0.3070 - val_accuracy: 0.9743
Epoch 2915/3000
183/183 [==============================] - 8s 43ms/step - loss: 0.0448 - accuracy: 0.9927 - val_loss: 0.3069 - val_accuracy: 0.9743
Epoch 2916/3000
183/183 [==============================] - 8s 43ms/step - loss: 0.0448 - accuracy: 0.9928 - val_loss: 0.3069 - val_accuracy: 0.9743
Epoch 2917/3000
183/183 [==============================] - 8s 42ms/step - loss: 0.0490 - accuracy: 0.9929 - val_loss: 0.3131 - v

183/183 [==============================] - 8s 44ms/step - loss: 0.0428 - accuracy: 0.9925 - val_loss: 0.3053 - val_accuracy: 0.9712
Epoch 2967/3000
183/183 [==============================] - 8s 42ms/step - loss: 0.0448 - accuracy: 0.9919 - val_loss: 0.3060 - val_accuracy: 0.9684
Epoch 2968/3000
183/183 [==============================] - 8s 43ms/step - loss: 0.0441 - accuracy: 0.9874 - val_loss: 0.3116 - val_accuracy: 0.9677
Epoch 2969/3000
183/183 [==============================] - 8s 43ms/step - loss: 0.0428 - accuracy: 0.9865 - val_loss: 0.2833 - val_accuracy: 0.9660
Epoch 2970/3000
183/183 [==============================] - 8s 45ms/step - loss: 0.0400 - accuracy: 0.9849 - val_loss: 0.2833 - val_accuracy: 0.9633
Epoch 2971/3000
183/183 [==============================] - 8s 45ms/step - loss: 0.0407 - accuracy: 0.9840 - val_loss: 0.2831 - val_accuracy: 0.9636
Epoch 2972/3000
183/183 [==============================] - 8s 45ms/step - loss: 0.0400 - accuracy: 0.9838 - val_loss: 0.3052 - v